In [101]:
import os
import pandas as pd
import shutil
from PIL import Image, ImageSequence

In [110]:
df_metadata = pd.read_csv('metadata.csv',header=None)
df_metadata_test = pd.read_csv('metadata_test.csv',header=None)
input_folder_path='Received From Client - Copyinput'
test_folder_path = 'Received From Client - Copytest'
#unique check
if len(df_metadata_test[df_metadata_test[1].duplicated()==True][0])==0:
    print("metadata_test is not duplicated.")
if len(df_metadata[df_metadata[1].duplicated()==True][0])==0:
    print("metadata is not duplicated.")
#folder check
input_folder_list=os.listdir(input_folder_path)
test_folder_list=os.listdir(test_folder_path)
if len(input_folder_list+test_folder_list)==len(set(input_folder_list+test_folder_list)):
    print("No files are common in input and test")
#metadata with folder check
if len(df_metadata_test[df_metadata_test[1].isin(test_folder_list)==False])==0:
    print("all metadata coincides with test folder.")
if len(df_metadata[df_metadata[1].isin(input_folder_list)==False])==0:
    print("all metadata coincides with input folder.")
if len(df_metadata[df_metadata[1].isin(test_folder_list)==True])==0:
    print("no metadata coincides with test folder.")
if len(df_metadata_test[df_metadata_test[1].isin(input_folder_list)==True])==0:
    print("no metadata_test coincides with input folder.")
else:
    print(df_metadata_test[df_metadata_test[1].isin(input_folder_list)==True])

metadata_test is not duplicated.
No files are common in input and test
all metadata coincides with test folder.
all metadata coincides with input folder.
no metadata coincides with test folder.
no metadata_test coincides with input folder.


In [132]:
df_metadata = pd.read_csv('meta.csv',header=None)
input_folder_path='Received From Client - Copyinput'
#unique check
if len(df_metadata[df_metadata[1].duplicated()==True][0])==0:
    print("metadata is not duplicated.")
else:
    print(df_metadata[df_metadata[1].duplicated()==True])
#folder check
input_folder_list=os.listdir(input_folder_path)
if len(input_folder_list)==len(set(input_folder_list)):
    print('No dupli in fol')
if len(df_metadata)==len(input_folder_list):
    print("Exactly same length.")
#metadata with folder check
print(df_metadata[df_metadata[1].isin(input_folder_list)==False])

       0                       1
270  EOB  089001563322 Bill1.pdf
272  EOB  089001563322 Bill4.pdf
274  EOB   089001563569 EOB1.pdf
276  EOB   089001563569 EOB2.pdf
278  EOB    089001579486 EOB.pdf
280  EOB    092001323227 EOB.pdf
282  EOB    092001597474 EOB.pdf
284  EOB    092001609159 EOB.pdf
286  EOB    092001742331 EOB.pdf
288  EOB    092001978741 EOB.pdf
No dupli in fol
Exactly same length.
Empty DataFrame
Columns: [0, 1]
Index: []


In [5]:
def tiff_to_pdf(tiff_path: str) -> str:
 
    pdf_path = tiff_path.replace('.tif', '.pdf')
    
#     if not os.path.exists(tiff_path): raise Exception(f'{tiff_path} does not find.')
    image = Image.open(tiff_path)
 
    images = []
    for i, page in enumerate(ImageSequence.Iterator(image)):
        page = page.convert("RGB")
        images.append(page)
    if len(images) == 1:
        print(pdf_path)
        images[0].save(pdf_path)
    else:
        images[0].save(pdf_path, save_all=True,append_images=images[1:])
    return pdf_path

In [124]:
def copy_file(source_path):
    path = source_path
    folder_name=path.split('\\')[-1]
    destination_path = source_path+'input'
    removed_file_size=[]
    removed_file_ext=[]
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    file_list=[]
    class_list=[]
    for root,dirs,files in os.walk(source_path):
        for file in files:
            path_file = root+"\\"+file
            dest_file = destination_path+"\\"+file
            file_ext = file.split('.')[-1]
            if os.stat(path_file).st_size>54525952:
                os.remove(path_file)
                removed_file_size.append(file)
                print(f'{file} is removed because of more size than 52mb.')
            elif file_ext != 'tif' and file_ext!='pdf' and file_ext!='jpg' and file_ext!='jpeg' and file_ext!='JPG' and file_ext!='JPG' and file_ext!='tiff' and file_ext!='TIF' and file_ext!='TIFF' and file_ext!='tiff':
                os.remove(path_file)
                removed_file_ext.append(file)
                print(f'{file} is removed because {file_ext} is not supported.')
            else:
                if file.endswith('.tif') or file.endswith('.tiff') or file.endswith('.TIF') or file.endswith('.TIFF'):
                    tiff_to_pdf(path_file)
                    os.remove(path_file)
                    file = file.replace('tif','pdf')
                    path_file = root+"\\"+file
                shutil.copyfile(path_file,dest_file)
                print("file",file)
                file_list.append(file)
                class_list.append(root.split('\\')[root.split('\\').index(folder_name)+1])
    print("All the count files are moved to desti folder.")
    return file_list,class_list,removed_file_size,removed_file_ext

In [106]:
def copy_file_ratio(source_path, test_ratio):
    dest_path_input = source_path+'input'
    dest_path_test = source_path+'test'
    if not os.path.exists(dest_path_input):
        os.makedirs(dest_path_input)
    if not os.path.exists(dest_path_test):
        os.makedirs(dest_path_test)    
    file_list=[]
    class_list=[]
    file_list_test=[]
    class_list_test=[]
    removed_file_size=[]
    removed_file_ext=[]
    for aclass in os.listdir(source_path):
        nu_of_file_class=0
        source_path_class = source_path+'\\'+aclass
        file_list_ori = []
        for root,dirs,files in os.walk(source_path_class):
            for file in files:
                file_list_ori.append(file)
                nu_of_file_class+=1
        if nu_of_file_class!=0:
            nu_files_to_input = int((nu_of_file_class*test_ratio)/100)
            files_input = file_list_ori[:nu_files_to_input]  
            files_test = file_list_ori[nu_files_to_input:]
            file_tot=files_input+files_test
            file_tot_set=set(file_tot)
            for root, dirs, files in os.walk(source_path_class):
                for file in files:
                    path_file = root+"\\"+file
                    file_ext = file.split('.')[-1]
                    if os.stat(path_file).st_size>54525952:
                        os.remove(path_file)
                        removed_file_size.append(file)
                        print(f'{file} is removed because of more size than 52mb.')
                    elif file_ext != 'tif' and file_ext!='pdf' and file_ext!='jpg' and file_ext!='jpeg' and file_ext!='JPG' and file_ext!='JPG' and file_ext!='tiff' and file_ext!='TIF' and file_ext!='TIFF' and file_ext!='tiff':
                        os.remove(path_file)
                        removed_file_ext.append(file)
                        print(f'{file} is removed because {file_ext} is not supported.')
                    else:
                        if file in files_input:
                            if file.endswith('.tif') or file.endswith('.tiff') or file.endswith('.TIF') or file.endswith('.TIFF'):
                                tiff_to_pdf(path_file)
                                os.remove(path_file)
                                file = file.replace('tif','pdf')
                                path_file = path_file.replace('tif','pdf')
                            dest_file = dest_path_input+"\\"+file
                            file_list.append(file)
                            class_list.append(aclass)
                        elif file in files_test:
                            if file.endswith('.tif') or file.endswith('.tiff') or file.endswith('.TIF') or file.endswith('.TIFF'):
                                tiff_to_pdf(path_file)
                                os.remove(path_file)
                                file = file.replace('tif','pdf')
                                path_file = path_file.replace('tif','pdf')
                            dest_file = dest_path_test+"\\"+file                            
                            file_list_test.append(file)
                            class_list_test.append(aclass)
                        shutil.copyfile(path_file,dest_file)
    return file_list,class_list, file_list_test,class_list_test,removed_file_size,removed_file_ext

In [125]:
source_path = "Received From Client - Copy"
test_ratio=0
if test_ratio==0:
    file_list,class_list,removed_file_size,removed_file_ext = copy_file(source_path)
    if len(file_list)==len(class_list):
        df=pd.DataFrame(file_list,class_list).reset_index()
        df.to_csv("meta.csv",index=False,header=False)
        print("Metadata is successfully saved to meta.csv file.")
    else:
        print("there is some discrepency in the folder.")
else:
    file_list_input,class_list_input,file_list_test,class_list_test,removed_file_size,removed_file_ext = copy_file_ratio(source_path,test_ratio)
    if len(file_list_input)==len(class_list_input):
        df=pd.DataFrame(file_list_input,class_list_input).reset_index()
        df.to_csv("metadata.csv",index=False,header=False)
        print("Input Metadata is successfully saved to metadata.csv file.")
    else:
        print("there is some discrepency in the input folder.")
    if len(file_list_test)==len(class_list_test):
        df_test=pd.DataFrame(file_list_test,class_list_test).reset_index()
        df_test.to_csv("metadata_test.csv",index=False,header=False)
        print("Test Metadata is successfully saved to metadata_test.csv file.")
    else:
        print("there is some discrepency in the test folder.")
print('List of removed file because of size more than 52mb:',removed_file_size)
print('List of removed file because of extension:')
for i in removed_file_ext:
    print(i,i.split('.')[-1])


file 0097aftervisitsummary.pdf
file 0297aftervisitsummary.pdf
file 3227aftervisitsummary.pdf
file 3667aftervisitsummary.pdf
file 507aftervisitsummary.pdf
file 5117aftervisitsummary.pdf
file 5327aftervisitsummary.pdf
file 5647aftervisitsummary.pdf
file Admission_Discharge Summary 5.pdf
file After visit summary 2.pdf
file after visit summary.pdf
file after visit summary1 - Copy.pdf
file after visit summary1.pdf
file afv - Copy.pdf
file afv.pdf
file AVS 5 - Copy.pdf
file AVS 5.pdf
file AVS 6- After visit summary - Copy.pdf
file AVS 6- After visit summary.pdf
file AVS 7 - Copy.pdf
file AVS 7.pdf
file AVS 8 - Copy.pdf
file AVS 8.pdf
file Doctor Visit 1 - Copy.pdf
file Doctor Visit 1.pdf
file Doctor Visit 2 - Copy.pdf
file Doctor Visit 2.pdf
file DV 1 - Copy.pdf
file DV 1.pdf
file DV 3 - Copy.pdf
file DV 3.pdf
file 089001564979 APS.pdf
file 089001575133 APS.pdf
file 089001612266 APS.pdf
file 092001504440 APS.pdf
file 092001601229 APS.pdf
file 092001850464 APS.pdf
file 092001862025 APS.pdf
fi

file 32351 ADMDSCHG (red).pdf
file 48579-Admission Discharge.pdf
file 69018 Admission Discharge.pdf
file 73179 ADMDSCHG (red).pdf
file 77391 Admission Discharge 2.pdf
file 77391 Admission Discharge.pdf
file 98769 Admission and Discharge.pdf
file AD 1- Admission and Discharge 2.pdf
file AD 2.pdf
file AD 3- Admission Records.pdf
file AD 4-Admission and Discharge.pdf
file AD 6 - Copy.pdf
file AD 6.pdf
file AD-1 Admission and Discharge - Copy.pdf
file AD-1 Admission and Discharge 2 - Copy.pdf
file AD-1 Admission and Discharge 2.pdf
file AD-1 Admission and Discharge.pdf
file Admission Discharge 11- Admission Discharge - Copy.pdf
file Admission Discharge 11- Admission Discharge.pdf
file Admission Discharge 6-Admission Discharge - Copy.pdf
file Admission Discharge 6-Admission Discharge.pdf
file Admission_Discharge Summary 10 - Copy.pdf
file Admission_Discharge Summary 10.pdf
file Admission_Discharge Summary 7 - Copy.pdf
file Admission_Discharge Summary 7.pdf
file Admission_Discharge Summary 8

file path 2.pdf
file path 3.pdf
file Path 4.pdf
file path 5.pdf
file Path.pdf
file 078001931001 Police Report - Copy.pdf
file 078001931001 Police Report.pdf
file 110000137288 Police Report - Copy.pdf
file 110000137288 Police Report.pdf
file Belmont Crash (red).pdf
file Belmont Crash - Copy.pdf
file Belmont Crash.pdf
file Police Report 2.pdf
file Police Report 3.pdf
file Police Report 4.pdf
file Police Report 5.pdf
file police report 6.pdf
file Police Report Example 1.pdf
file Police Report Example 10.pdf
file Police Report Example 11.pdf
file Police Report Example 2 - Copy.pdf
file Police Report Example 2.pdf
file Police Report Example 3.pdf
file Police Report Example 4.pdf
file Police Report Example 5.pdf
file Police Report Example 6.pdf
file Police Report Example 7 - Copy.pdf
file Police Report Example 7.pdf
file Police Report Example 8 - Copy.pdf
file Police Report Example 8.pdf
file Police Report Example 9 - Copy.pdf
file Police Report Example 9.pdf
file Police Report Sample 1 - Co

In [69]:
l=['a','b','c','d','e','f','g']

In [77]:
root.split('\\')[root.split('\\').index('Received From Client - Copy')+1]

NameError: name 'root' is not defined